# Setup
Install the necessary dependencies with `pip install ipython-sql`

In [20]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# How to submit
You are welcome to use this notebook to interactive try out different SQL queries.

To assess whether your queries are correct you will need to complete the 3 submission files provided.

Refer to the `README.md` for background on the dataset.

In [52]:
query = '''
SELECT *
FROM businesses
''' 
engine = create_engine("sqlite:///data/sfscores.sqlite")
df = pd.read_sql_query(query, engine)
df.head()

,business_id,name,address,city,postal_code,latitude,longitude,phone_number,TaxCode,business_certificate,application_date,owner_name,owner_address,owner_city,owner_state,owner_zip
0,10,Tiramisu Kitchen,033 Belden Pl,San Francisco,94104,37.791116,-122.403816,NaN,H24,779059.0,None,Tiramisu LLC,33 Belden St,San Francisco,CA,94104
1,19,Nrgize Lifestyle Cafe,"1200 Van Ness Ave, 3rd Floor",San Francisco,94109,37.786848,-122.421547,NaN,H24,NaN,None,24 Hour Fitness Inc,"1200 Van Ness Ave, 3rd Floor",San Francisco,CA,94109
2,24,OMNI S.F. Hotel - 2nd Floor Pantry,"500 California St, 2nd Floor",San Francisco,94104,37.792888,-122.403135,NaN,H24,352312.0,None,OMNI San Francisco Hotel Corp,"500 California St, 2nd Floor",San Francisco,CA,94104
3,31,Norman's Ice Cream and Freezes,2801 Leavenworth St,San Francisco,94133,37.807155,-122.419004,NaN,H24,346882.0,None,Norman Antiforda,2801 Leavenworth St,San Francisco,CA,94133
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM St,S.F.,94110,37.747114,-122.413641,NaN,H24,340024.0,2001-10-10,"HARB, CHARLES AND KRISTIN",1150 SANCHEZ,S.F.,CA,94114


# Queries

## Pt1: Essentials

**1. Write a SQL query that finds the number of business ids in the businesses table**

In [46]:
engine = create_engine("sqlite:///data/sfscores.sqlite")
query = '''
SELECT COUNT (business_id)
FROM businesses
''' 
result = engine.execute(query)
(n_bus_id, ) = result.fetchall()[0]
n_bus_id

7619

**2. Write a SQL query that finds out how many unique business names are registered with San Francisco Food health investigation organization and name the column as unique restaurant name count.**

In [54]:
df.head(1)

,business_id,name,address,city,postal_code,latitude,longitude,phone_number,TaxCode,business_certificate,application_date,owner_name,owner_address,owner_city,owner_state,owner_zip
0,10,Tiramisu Kitchen,033 Belden Pl,San Francisco,94104,37.791116,-122.403816,NaN,H24,779059.0,None,Tiramisu LLC,33 Belden St,San Francisco,CA,94104


In [66]:
flag = df['city'] == 'San Francisco'
df['name'][flag].nunique()

1808

In [69]:
query = '''
SELECT COUNT (DISTINCT name) as "unique resturant name count"
FROM businesses
'''
result = engine.execute(query)
result.fetchall()

[(7060,)]

**3. Write a SQL query that finds out what is the earliest and latest date a health investigation is recorded in this database.**

In [77]:
query = '''
SELECT min(date), max(date)
FROM inspections
'''
result = engine.execute(query)
result.fetchall()

[(20131007, 20161215)]

**4. How many businesses are there in San Francisco where their owners live in the same area (postal code/ zip code) as the business is located?**

In [82]:
query = '''
SELECT DISTINCT COUNT (*)
FROM businesses
WHERE postal_code = owner_zip
'''
result = engine.execute(query)
result.fetchall()

[(4053,)]

**5. Out of those businesses, how many of them has a registered business certificate?**

In [85]:
query = '''
SELECT DISTINCT COUNT (*)
FROM businesses
WHERE 
    postal_code = owner_zip AND
    business_certificate IS NOT NULL
'''
result = engine.execute(query)
result.fetchall()

[(3373,)]

## Pt2: Groupby

**6. Find out the distribution of the risk exposure of all the violations reported in the database. The first column of the result should 'risk_category' and the second column the count.**

In [88]:
query = '''
SELECT risk_category, COUNT (date)
FROM violations
GROUP BY risk_category
'''
result = engine.execute(query)
result.fetchall()

[('High Risk', 5608), ('Low Risk', 20996), ('Moderate Risk', 14131)]

**7. Find out the distribution of the risk exposure of all the violations reported in the database that are *water related*. Sort them by frequency (count) from high to low.**

In [99]:
query = '''
SELECT risk_category, COUNT(date) as frequency
FROM violations
WHERE
    description like '%water%'
GROUP BY risk_category
ORDER BY frequency DESC
'''
result = engine.execute(query)
result.fetchall()

[('Moderate Risk', 630), ('High Risk', 237)]

**8. What types of inspections does the authorities conduct and how often do they occur in general.
    Calculate the distribution of different types of inspections with their frequency (type, frequency)
    based on inspections records. Sort them in ascending order based on frequency.**

In [101]:
query = '''
SELECT type, COUNT(date) as frequency
FROM inspections
GROUP BY type
ORDER BY frequency ASC
'''
result = engine.execute(query)
result.fetchall()

[('Multi-agency Investigation', 1),
 ('Administrative or Document Review', 3),
 ('Special Event', 7),
 ('Routine - Scheduled', 74),
 ('Foodborne Illness Investigation', 80),
 ('Complaint Reinspection/Followup', 138),
 ('Structural Inspection', 170),
 ('Non-inspection site visit', 814),
 ('Complaint', 1625),
 ('New Construction', 1737),
 ('New Ownership', 1843),
 ('Reinspection/Followup', 5283),
 ('Routine - Unscheduled', 15568)]

**9. What is the average score given to restaurants based on the type of inspection?
    Based on the results, identify the types of inspections that are not scored (NULL)
    and remove those categories from the resultset. The 'average_score' should be rounded
    to one decimal. Sort the results in ascending order based on the average score. Hint: use the function `ROUND(score, 1)`**

In [108]:
query = '''
SELECT type, round(AVG(Score),1) as average_score
FROM inspections
WHERE inspections.type = 'New Ownership' or inspections.type = 'Reinspection/Followup' or inspections.type = 'Routine - Unscheduled'
GROUP BY type
ORDER BY average_score ASC
'''
result = engine.execute(query)
result.fetchall()

[('Reinspection/Followup', 85.0),
 ('Routine - Unscheduled', 91.2),
 ('New Ownership', 100.0)]

**10. Find the restaurant owners (owner_name) that own one or multiple restaurants in the city
    with the number of restaurants (num_restaurants) they own.
    Find the first top 10 owners ordered by descending order using the number of restaurants.**

In [119]:
query = '''
SELECT owner_name, COUNT (name) AS num_restaurants
FROM businesses
GROUP BY owner_name
ORDER BY num_restaurants DESC
LIMIT 10
'''
result = engine.execute(query)
result.fetchall()

[('VOLUME SERVICES AMERICA', 94),
 ('Department of Children, Youth and their Families', 64),
 ('SFUSD', 44),
 ('Volume Services, Inc.', 41),
 ("Children's Empowerment Inc.", 37),
 ('SAN FRANCISCO UNIFIED SCHOOL', 34),
 ('WALGREEN CO.', 17),
 ('SMG Food and Beverage LLC', 15),
 ('Walgreen Co', 15),
 ('Walgreen Co.', 14)]

## Pt3: Subqueries and joins

**11. From the businesses table, select the top 10 most popular postal_code. They should be filtered to only count the restaurants owned by people/entities that own 5 or more restaurants. The final result should return a row (postal_code, count) for each 10 selection and be sorted by descending order to get the most relevant zip codes**

In [140]:
query = '''
    SELECT owner_name, COUNT(name) as count
    FROM businesses
    GROUP BY owner_name
    HAVING COUNT (name) >= 5
    ORDER BY count DESC
'''
result = engine.execute(query)
result.fetchall()

[('VOLUME SERVICES AMERICA', 94),
 ('Department of Children, Youth and their Families', 64),
 ('SFUSD', 44),
 ('Volume Services, Inc.', 41),
 ("Children's Empowerment Inc.", 37),
 ('SAN FRANCISCO UNIFIED SCHOOL', 34),
 ('WALGREEN CO.', 17),
 ('SMG Food and Beverage LLC', 15),
 ('Walgreen Co', 15),
 ('Walgreen Co.', 14),
 ('STARBUCKS COFFEE CO.', 13),
 ('Starbucks Corporation', 13),
 ('Bon Appetit Management Company', 11),
 ('San Francisco Unified School District', 11),
 ('Starbucks Coffee Co', 11),
 ('Au Energy, LLC', 10),
 ('Garfield Beach CVS, LLC', 10),
 ('San Francisco Bay Concessionaires', 10),
 (None, 9),
 ('Bon Appetit Management Co', 9),
 ('RA CENTER LLC', 9),
 ('San Francisco Soup Company', 9),
 ('JUMA VENTURES, INC.', 8),
 ('STARBUCKS COFFEE COMPANY', 8),
 ('Target Corporation', 8),
 ("Children's Empowerment, Inc.", 7),
 ('Guckenheimer Services, LLC', 7),
 ('MARRIOTT HOTELS, INC.', 7),
 ('S.F. Unified School District', 7),
 ('SAFEWAY STORES, INC.', 7),
 ('Volume Services, Inc

In [142]:
query = '''
SELECT postal_code, COUNT(*) as freq
FROM businesses as b
WHERE b.owner_name in 
    (SELECT owner_name
    FROM businesses
    GROUP BY owner_name
    HAVING COUNT (name) >= 5
    ORDER BY COUNT(name) DESC)
GROUP BY postal_code
ORDER BY freq DESC
LIMIT 10
'''
result = engine.execute(query)
result.fetchall()

[('94107', 179),
 ('94103', 68),
 ('94102', 56),
 ('94110', 32),
 ('94112', 30),
 ('94104', 28),
 ('94124', 27),
 ('94105', 23),
 ('94133', 22),
 ('94111', 21)]

**12. First let's get an idea about the inspection score our competition has.
    Based on multiple inspections, find out the minimum Score (as "min_score"),
    average Score (as "avg_score") and maximum Score (as "max_score") for all restaurant in post code "94103".
    The average score should be rounded to one decimal.**

In [148]:
query = '''
SELECT MIN(Score) as min_score, round(AVG(Score),1) as avg_score, MAX(SCore) as max_score
FROM inspections as i
    LEFT OUTER JOIN businesses as b
    ON i.business_id = b.business_id
WHERE
    b.postal_code = 94103
'''

result = engine.execute(query)
result.fetchall()

[(54, 90.2, 100)]

** 13. Now lets get more serious, and look at how many times restaurants in Market street
    (postal_code: 94103) has committed health violations and group them based on their risk category.
    The output should be (risk_category, count as frequency) and sorted in descending order by frequency**

In [154]:
query = '''
SELECT risk_category, COUNT(*) as frequency
FROM violations as v
    LEFT OUTER JOIN businesses as b
    ON v.business_id = b.business_id
WHERE b.postal_code = 94103
GROUP BY risk_category
ORDER BY frequency DESC
'''
result = engine.execute(query)
result.fetchall()

[('Low Risk', 1746), ('Moderate Risk', 1229), ('High Risk', 520)]